# Website database management

This is temporary Jupyter Notebook/Dashboard for routine database management task. Use this as a trial for various tasks.  
When tasks are mature, move them to python scripts that can be run as a cron job.

In [1]:
import sqlite3
from pathlib import Path

In [2]:
#    Constants
# -------------------------------------------------------------------- #

DB_FOLDER = Path('../')
DB_NAMES = {
    'default_db': 'db.sqlite3',
    'old_messages_db': 'db_old_messages.sqlite3',
    'analytics_db': 'db_analytics.sqlite3',
}

## Create functions

Note that `conn` and `cursor` would not live outside the function (true for Python, although sqlite file handling may bring an exception). So it's (probably) safe to use these functions freely at any time without worrying about existing connections called `conn` or `cursor`. 

In [3]:
# create a function to display a list of matching tables in a database
def show_tables(db_path, matching:str = ''):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name LIKE '%" + matching + "%'")
    tables_list = cursor.fetchall()
    conn.close()
    return tables_list

# show the list of all columns in the table
def show_columns(db_path, table_name):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM " + table_name)
    columns_list = [description[0] for description in cursor.description]
    conn.close()
    return columns_list

## Test functions

### Show tables in a database

In [5]:
# show first five tables in main db matching base
show_tables((DB_FOLDER / DB_NAMES['default_db']), 'base')[:5]

[('base_proposal',),
 ('base_teaching',),
 ('base_workhighlight',),
 ('base_skillset',),
 ('base_honorandaward',)]

### Show columns in a table

In [6]:
# show columns in table base_message
show_columns((DB_FOLDER / DB_NAMES['default_db']), 'base_message')

['id',
 'senders_name',
 'senders_email',
 'message',
 'timestamp',
 'email_in_success',
 'email_out_success',
 'hidden_field']

In [7]:
# show columns in table base_analytic
show_columns((DB_FOLDER / DB_NAMES['default_db']), 'base_analytic')

['id',
 'timestamp',
 'content_type',
 'http_accept_language',
 'http_host',
 'http_referer',
 'http_user_agent',
 'remote_host',
 'remote_user',
 'remote_port',
 'request_method',
 'server_name',
 'server_port',
 'remote_addr',
 'visited_page']

## Tasks

### Move messages to a different database

In order to keep the default Django database small, we will periodically move messages to a different database. What's a good way to do this?
- Because we read messages from the default database, it's not a good idea to move all messages out form there. Especially if we do this as a cron job, we won't have time to read the messages before they are moved.
- Plan to keep most recent 60 days or so worth of messages in the default database.
- Move older or all messages to a different database. Just make sure to not make duplicate entries.

In [17]:
#    Connect to both databases
# -------------------------------------------------------------------- #

# create a database to store old messages 
# NOTE: this will be called `main`
conn = sqlite3.connect((DB_FOLDER / DB_NAMES['old_messages_db']))

# attach the default database to the connection (avoiding using `main` as a keyword as that seems to be a reserved word in sqlite3)
conn.execute("ATTACH DATABASE '" + str(DB_FOLDER / DB_NAMES['default_db']) + "' AS default_db")

# create a copy of the table base_message without data
conn.execute("CREATE TABLE IF NOT EXISTS main.base_message AS SELECT * FROM default_db.base_message WHERE 0")

cursor = conn.cursor()

# just for fun, print the databases the cursor is connected to
cursor.execute("PRAGMA database_list")
print(cursor.fetchall())

# check that the old_messages database contains the base_message table
# NOTE: the function show_tables() will not work here because it uses a different connection and the db file hasn't actually been saved yet
print()
print('Tables found in old_messages_db:')
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
print(cursor.fetchall())

# print the last 5 rows of the main.base_message table
print()
print('Last 5 rows of main.base_message:')
cursor.execute("SELECT * FROM main.base_message ORDER BY id DESC LIMIT 5")
print(cursor.fetchall())

# print the number of rows in the main.base_message table
print()
cursor.execute("SELECT COUNT(*) FROM main.base_message")
print('main.base_message contains', cursor.fetchone()[0], 'rows')

# print the number of rows in the default.base_message table
cursor.execute("SELECT COUNT(*) FROM default_db.base_message")
print('default_db.base_message contains', cursor.fetchone()[0], 'rows')

[(0, 'main', 'd:\\OneDrive\\Development\\Sites\\prateekverma\\old_messages.sqlite3'), (2, 'default_db', 'd:\\OneDrive\\Development\\Sites\\prateekverma\\db.sqlite3')]

Tables found in old_messages_db:
[('base_message',)]

Last 5 rows of main.base_message:
[]

main.base_message contains 0 rows
default_db.base_message contains 7456 rows


In [19]:
#    Move old messages to old_messages_db
# -------------------------------------------------------------------- #

# copy entries older than 60 days from default_db.base_message to main.base_message if they don't already exist
cursor.execute("INSERT INTO main.base_message SELECT * FROM default_db.base_message WHERE id NOT IN (SELECT id FROM main.base_message) AND timestamp < date('now', '-60 day')")

# delete entries in default_db.base_message already in main.base_message and (just for safety) older than 60 days
cursor.execute("DELETE FROM default_db.base_message WHERE id IN (SELECT id FROM main.base_message) AND timestamp < date('now', '-60 day')")

# print the number of rows in the main.base_message table
cursor.execute("SELECT COUNT(*) FROM main.base_message")
print('main.base_message contains', cursor.fetchone()[0], 'rows')

# print the number of rows in the default.base_message table
cursor.execute("SELECT COUNT(*) FROM default_db.base_message")
print('default_db.base_message contains', cursor.fetchone()[0], 'rows')

main.base_message contains 7456 rows
default_db.base_message contains 0 rows


In [20]:
#    Commit and save changes
# -------------------------------------------------------------------- #

# commit changes and close the connection
conn.commit()
conn.close()

That's it. Some things we can do further:
- prune the old messages database for spam messages periodically. We can just auto delete spams older than 60 days from the latest timestamp in the database. Or we can prune the entire databse of spams manually.
- we can prepare and send a digest weekly/monthly, if we ever set this as a cron job.

### Move analytics to a different database

We don't need to keep any analytics in the default database. Whenever we run this script, whether manually or periodically through cron, we can move all analytics to a different database.

In [21]:
#    Connect to both databases
# -------------------------------------------------------------------- #

# create a database to store analytics
# NOTE: this will be called `main`
conn = sqlite3.connect((DB_FOLDER / DB_NAMES['analytics_db']))

# attach the default database to the connection (avoiding using `main` as a keyword as that seems to be a reserved word in sqlite3)
conn.execute("ATTACH DATABASE '" + str(DB_FOLDER / DB_NAMES['default_db']) + "' AS default_db")

# create a copy of the table base_analytic without data
conn.execute("CREATE TABLE IF NOT EXISTS main.base_analytic AS SELECT * FROM default_db.base_analytic WHERE 0")

cursor = conn.cursor()

# just for fun, print the databases the cursor is connected to
cursor.execute("PRAGMA database_list")
print(cursor.fetchall())

# check that the analytics database contains the base_analytic table
# NOTE: the function show_tables() will not work here because it uses a different connection and the db file hasn't actually been saved yet
print()
print('Tables found in analytics_db:')
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
print(cursor.fetchall())

# print the last 5 rows of the main.base_analytic table
print()
print('Last 5 rows of main.base_analytic:')
cursor.execute("SELECT * FROM main.base_analytic ORDER BY id DESC LIMIT 5")
print(cursor.fetchall())

# print the number of rows in the main.base_analytic table
print()
cursor.execute("SELECT COUNT(*) FROM main.base_analytic")
print('main.base_analytic contains', cursor.fetchone()[0], 'rows')

# print the number of rows in the default.base_analytic table
cursor.execute("SELECT COUNT(*) FROM default_db.base_analytic")
print('default_db.base_analytic contains', cursor.fetchone()[0], 'rows')

[(0, 'main', 'd:\\OneDrive\\Development\\Sites\\prateekverma\\analytics.sqlite3'), (2, 'default_db', 'd:\\OneDrive\\Development\\Sites\\prateekverma\\db.sqlite3')]

Tables found in analytics_db:
[('base_analytic',)]

Last 5 rows of main.base_analytic:
[]

main.base_analytic contains 0 rows
default_db.base_analytic contains 42625 rows


In [22]:
#    Move analytics to analytics_db
# -------------------------------------------------------------------- #

# copy entries from default_db.base_analytic to main.base_analytic if they don't already exist
cursor.execute("INSERT INTO main.base_analytic SELECT * FROM default_db.base_analytic WHERE id NOT IN (SELECT id FROM main.base_analytic)")

# delete entries in default_db.base_analytic already in main.base_analytic
cursor.execute("DELETE FROM default_db.base_analytic WHERE id IN (SELECT id FROM main.base_analytic)")

# print the number of rows in the main.base_analytic table
cursor.execute("SELECT COUNT(*) FROM main.base_analytic")
print('main.base_analytic contains', cursor.fetchone()[0], 'rows')

# print the number of rows in the default.base_analytic table
cursor.execute("SELECT COUNT(*) FROM default_db.base_analytic")
print('default_db.base_analytic contains', cursor.fetchone()[0], 'rows')

main.base_analytic contains 42625 rows
default_db.base_analytic contains 0 rows


In [23]:
#    Commit and save changes
# -------------------------------------------------------------------- #

# commit changes and close the connection
conn.commit()
conn.close()

### Vacuum database to reclaim space

We can run this periodically to reclaim space if large amount of data is deleted. In a routine scenario, this won't be needed as the database file size will remain small and approximately the same.

In [24]:
# vacuum the default database to reduce file size
conn = sqlite3.connect((DB_FOLDER / DB_NAMES['default_db']))
conn.execute("VACUUM")
conn.close()